In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 라이브러리 설치 및 버전 확인

In [ ]:
# !pip install numpy==1.19.5
# !pip install matplotlib==3.4.0
# !pip install scikit-learn==0.23.2
# !pip install pycaret==2.3.5


In [ ]:
# from pycaret.classification import *
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import kurtosis

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, log_loss, accuracy_score

from sklearn.decomposition import PCA
import plotly.express as px


# 데이터 불러오기

- 훈련 데이터와 테스트 데이터는 16개 컬럼으로 구성 (독립변수 16개)
- 종속 변수는 1개 ('state') -> train_labels에 들어가 있음
- 모두 숫자형 데이터
- sequnce를 기준으로 train과 train_labels 데이터 프레임 병합

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
train_labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')
    
print("Train_shape: ", train.shape)
print("Test_shape: ", test.shape)
print("Train_labels_shape: ", train_labels.shape)
print("Submission_shape: ", submission.shape)

In [ ]:
print(train.info())
print('')
print(train_labels.info())

In [ ]:
train = pd.merge(train,train_labels,on='sequence')

In [ ]:
train.head(10)

# 결측치 확인
  - 훈련 데이터셋과 테스트 데이터셋 모두 결측치가 없음

In [ ]:
print('Missing Values of Train Data:')
print(train.isna().sum())
print('Missing Values of Test Data:')
print(test.isna().sum())

# EDA

- 타겟 데이터 분포 확인
    + 매우 고르다는 것을 확인

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(10, 6))
sns.countplot(data=train, x='state', ax=ax);

- 독립변수와 종속변수간 연관성 확인

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))

sns.kdeplot(data=train,x='subject', hue='state', ax=ax);
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.countplot(data=train,x='step', hue='state', ax=ax);
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.kdeplot(data=train,x='sequence', hue='state', ax=ax);
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
sns.kdeplot(data=train, x='sequence', shade=True, bw=10,color='black',ax=axes[0])
sns.kdeplot(data=train, x='subject', shade=True,bw=10,color='red', ax=axes[1])
sns.kdeplot(data=train, x='step', shade=True, bw=10,color='green', ax=axes[2]);
plt.tight_layout()
plt.show()

- 'sequence', 'step', 'subject'는 정규분포로 보인다

In [ ]:
figure = plt.figure(figsize=(16, 8))
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    plt.subplot(4, 4, sensor+1)
    plt.hist(train[train.state.isnull() == False][f"{sensor_name}"], bins=100)
    plt.title(f"{sensor_name}")
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.suptitle('Sensor Normal Probability Plot', y=1.02)
plt.show()

- 해석
    + 대부분이 0값에 집중되어 있음
    + sensor2를 빼고 대부분 정규분포 형태..?

- 상관관계 분석

In [ ]:
'''
def display_p_values(df, columns, target, th=0.05, cut=False):
    from scipy.stats import pearsonr
    p_values_list = []
    for c in columns:
        p = round(pearsonr(train.loc[:,target], train.loc[:,c])[1], 4)
        p_values_list.append(p)

    p_values_df = pd.DataFrame(p_values_list, columns=[target], index=columns)
    def p_value_warning_background(cell_value):
        highlight = 'background-color: lightcoral;'
        default = ''
        if cell_value > th:
                return highlight
        return default
    
    if cut:
        p_values_df_high = p_values_df[p_values_df[target] > th]
    else:
        p_values_df_high = p_values_df.copy()
    display(p_values_df_high.style.applymap(p_value_warning_background))
'''

In [ ]:
'''
sensor_cols = ['sensor_'+'%02d'%i for i in range(0, 13)]
sensor_cols.append('step')
display_p_values(train, sensor_cols, 'state', th=0.05)
'''

- 'sensor 02,04,10'은 'state'와 상관관계가 거의 없다

# Feature Engineering

- 각 스텝별 센서들의 통계값

In [ ]:
from scipy.stats.mstats import gmean
sensor = ['00','01','02','03','04','05','06','07','08','09','10','11','12']
for i in sensor:
    train[f'sensor_{i}_step_diff'] = train.groupby(['sequence','subject'])[f'sensor_{i}'].diff()
    train[f'sensor_{i}_step_diff'].fillna(train[f'sensor_{i}'].median(), inplace = True)
    train[f'sensor_{i}_step_mean'] = train.groupby(['sequence','subject'])[f'sensor_{i}'].transform('mean')
    train[f'sensor_{i}_step_median'] = train.groupby(['sequence','subject'])[f'sensor_{i}'].transform('median')
    train[f'sensor_{i}_step_std'] = train.groupby(['sequence','subject'])[f'sensor_{i}'].transform('std')
    train[f'sensor_{i}_step_min'] = train.groupby(['sequence','subject'])[f'sensor_{i}'].transform('min')
    train[f'sensor_{i}_step_max'] = train.groupby(['sequence','subject'])[f'sensor_{i}'].transform('max')
    
    test[f'sensor_{i}_step_diff'] = test.groupby(['sequence','subject'])[f'sensor_{i}'].diff()
    test[f'sensor_{i}_step_diff'].fillna(test[f'sensor_{i}'].median(), inplace = True)
    test[f'sensor_{i}_step_mean'] = test.groupby(['sequence','subject'])[f'sensor_{i}'].transform('mean')
    test[f'sensor_{i}_step_median'] = test.groupby(['sequence','subject'])[f'sensor_{i}'].transform('median')
    test[f'sensor_{i}_step_std'] = test.groupby(['sequence','subject'])[f'sensor_{i}'].transform('std')
    test[f'sensor_{i}_step_min'] = test.groupby(['sequence','subject'])[f'sensor_{i}'].transform('min')
    test[f'sensor_{i}_step_max'] = test.groupby(['sequence','subject'])[f'sensor_{i}'].transform('max')

In [ ]:
display(train.head(2),test.head(2))

- 시계열의 특징을 고려하여 1-step만큼 지연(lag)시켜보았다 

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"    
    train[f"{sensor_name}" + '_lag1'] = train.groupby('sequence')[f"{sensor_name}"].shift(1)  
    train[f"{sensor_name}" + '_lag1'].fillna(0, inplace=True)
    
    test[f"{sensor_name}" + '_lag1'] = test.groupby('sequence')[f"{sensor_name}"].shift(1)  
    test[f"{sensor_name}" + '_lag1'].fillna(0, inplace=True)


In [ ]:
display(train.head(2),test.head(2))

- 여기서 test 데이터셋의 column이 107개로 train과 맞지 않는 문제 발생,,,

# PCA

In [ ]:
y=train['state'].copy().astype(int)
X=train.drop('state', axis=1).copy()
X_test=test.copy()
print(X.shape,y.shape)

In [ ]:
'''
X_missing_values = X.isna().sum()
print(X_missing_values.value_counts())
'''

In [ ]:
'''
pca = PCA(n_components=2)
components = pca.fit_transform(X)
print(components.shape)

total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter(components, x=0, y=1, color=y,
    title=f'Total Explained Variance: {total_var:.2f}%',
    width=800, height=500
)
fig.show()
'''

In [ ]:
'''
pca = PCA().fit(X)
fig, ax = plt.subplots(figsize=(10,4))
xi = np.arange(1, 1+X.shape[1], step=1)
yi = np.cumsum(pca.explained_variance_ratio_)
plt.plot(xi, yi, marker='o', linestyle='--', color='b')
# plt.ylim(0.0,1.1)
plt.xlabel('Number of Components')
# plt.xticks(np.arange(1, 1+X.shape[1], step=10))
plt.ylabel('Cumulative variance (%)')
plt.title('Explained variance by each component')
'''

# 모델 훈련

In [ ]:
all_df = pd.concat([train,test])
all_df.head()

In [ ]:
X = all_df[:train.shape[0]]
print("X Shape is:", X.shape)
y = X['state']
X.drop(['state'], axis=1, inplace=True)
test_data = all_df[train.shape[0]:].drop(columns=['state'])
test_data.info()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
sub_train, val_train, sub_test, val_test = train_test_split(
    X_train, y_train, test_size = 0.2, random_state = 42
)
print(sub_train.shape, val_train.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

lgb = LGBMClassifier(random_state=42)

splitter = StratifiedKFold(n_splits=3, shuffle = True, random_state=42)
scores = cross_validate(lgb, X_train, y_train, cv=splitter)
print(np.mean(scores['test_score']))

In [ ]:
lgb.fit(X_train, y_train)
y_pred = lgb.predict(X_val)

print("Acc :", accuracy_score(y_val, y_pred))

# 제출

In [ ]:
lgb_test= lgb.predict(test)
pred = pd.DataFrame({"sequence":test['sequence'],
              "state":lgb_test.tolist()})

In [ ]:
pred_fin = pred.groupby('sequence').mean().reset_index()
for i in pred_fin.index.to_list():
    if pred_fin.iloc[i,1] != 0:
        pred_fin.iloc[i,1] = 1
pred_fin['state'].value_counts()

In [ ]:
submission['state'] = pred_fin['state'].astype(int)
submission.info()

In [ ]:
submission.to_csv('submission.csv', index=False)

# 참조 코드
- https://www.kaggle.com/code/hasanbasriakcay/tpsapr22-eda-fe-baseline#Distribution
- https://www.kaggle.com/code/javigallego/tps-apr22-eda-fe-lstm-tutorial
- https://www.kaggle.com/code/abdulravoofshaik/important-features-and-insights